In [66]:
'''
Author: bob
Date: 2022-05-24 16:59:52
LastEditors: bob
LastEditTime: 2022-05-24 16:59:58
FilePath: \任务22-开发网络版对账单转换工具\main.ipynb
Description: 使用python编写的网络版对账单转换工具

Copyright (c) 2022 by bob, All Rights Reserved. 
'''


'\nAuthor: bob\nDate: 2022-05-24 16:59:52\nLastEditors: bob\nLastEditTime: 2022-05-24 16:59:58\nFilePath: \\任务22-开发网络版对账单转换工具\\main.ipynb\nDescription: 使用python编写的网络版对账单转换工具\n\nCopyright (c) 2022 by bob, All Rights Reserved. \n'

In [67]:
# open读取txt文件

with open('data.txt', 'r',encoding='utf-8') as f:
    data = f.read()
data

'212593 LBVHY3107MMZ64537 A5JVU6\n8700060086 0000105 32.74 80.00 2 52.38 6.81 59.19\n8700020086 0000610 32.74 80.00 3 78.58 10.22 88.80\n8700020086 11428593186 181.42 80.00 1 145.14 18.87 164.01\n8700020086 4 574.46 80.00 1 459.57 59.74 519.31\n_____________________________________________________________________________________________________________________________________\n* 赔付145.14 130.96 459.57 95.64 831.31\n213378 LBVKY9108LSX09222 A5NONU\n0012740500 0072182 31.90 100.00 14 446.60 58.06 504.66\n0012740500 12415A77FD6 36.28 100.00 29.00- 1 25.76 1.85 3.59 31.20\n0012740500 12415A78409 144.25 100.00 21.00- 1 113.96 8.21 15.88 138.05\n_____________________________________________________________________________________________________________________________________\n* 赔付139.72 446.60 10.06 77.53 673.91\n213377 LBVKY9109LSW76277 A5NONV\n0012740500 0072182 31.90 100.00 14 446.60 58.06 504.66\n0012740500 07147310674 18.58 100.00 29.00- 1 13.19 0.95 1.84 15.98\n0012740500 12415A77FD6

In [68]:
# 1.通过re模块提取数据-表头
import re
p=r'(\d{5,6}) (\S{17}) (\S{6})'
res_head=re.findall(p,data)
res_head


[('212593', 'LBVHY3107MMZ64537', 'A5JVU6'),
 ('213378', 'LBVKY9108LSX09222', 'A5NONU'),
 ('213377', 'LBVKY9109LSW76277', 'A5NONV'),
 ('213379', 'LBVKY9108LSU91847', 'A5NONW'),
 ('213380', 'LBVKY5101LSY22048', 'A5NONX'),
 ('213376', 'LBVKY9102LSX33807', 'A5NONZ'),
 ('213386', 'LBV6P4107LMS18233', 'A5NOO8'),
 ('213369', 'LBVKY9106KSR97952', 'A5NOOF'),
 ('213368', 'LBV61AF08NS331986', 'A5NOOG'),
 ('213367', 'LBV31DU00NS364715', 'A5NOOH'),
 ('213366', 'LBVKY1104JSR58339', 'A5NOOI'),
 ('213365', 'LBV6P4101MMZ13664', 'A5NOOJ'),
 ('213364', 'LBV31DU03NS331031', 'A5NOOK'),
 ('213362', 'LBV8W3100KMP31408', 'A5NOOL'),
 ('213360', 'LBV61AF09NS197957', 'A5NOOM'),
 ('213359', 'LBV7Y4108MS097226', 'A5NOON'),
 ('213357', 'LBV61AF08MS183742', 'A5NOOP'),
 ('213406', 'LBVKY9101JSP05459', 'A5NPF8'),
 ('213405', 'LBVKY9108LSU91377', 'A5NPF9'),
 ('213407', 'LBV61AF04MS029237', 'A5NPFA'),
 ('213408', 'LBVKY910XJSP06755', 'A5NPFB'),
 ('213409', 'LBVKY9104KSR88618', 'A5NPFC'),
 ('213404', 'LBVKY9101KSR88771',

In [69]:
# 2.通过re模块提取数据-表体
# import re
p=r'\*( 赔付)(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*) ?(\S*)'
res_body=re.findall(p,data)
res_body

[(' 赔付', '145.14', '130.96', '459.57', '95.64', '831.31', '', ''),
 (' 赔付', '139.72', '446.60', '10.06', '77.53', '673.91', '', ''),
 (' 赔付', '152.91', '446.60', '11.01', '79.37', '689.89', '', ''),
 (' 赔付', '139.72', '446.60', '10.06', '77.53', '673.91', '', ''),
 (' 赔付', '27,137.40', '2,424.40', '1,953.90', '4,097.04', '35,612.74', '', ''),
 (' 赔付', '152.91', '446.60', '11.01', '79.37', '689.89', '', ''),
 (' 赔付', '1,221.24', '478.50', '87.93', '232.39', '2,020.06', '', ''),
 (' 赔付', '20,495.92', '1,052.70', '1,475.71', '2,993.16', '26,017.49', '', ''),
 (' 赔付', '145.14', '130.96', '549.36', '107.32', '932.78', '', ''),
 (' 赔付', '145.14', '130.96', '546.91', '107.00', '930.01', '', ''),
 (' 赔付', '2,773.64', '414.70', '485.39', '477.60', '4,151.33', '', ''),
 (' 赔付', '145.14', '130.96', '550.57', '107.47', '934.14', '', ''),
 (' 赔付', '145.14', '130.96', '550.57', '107.47', '934.14', '', ''),
 (' 赔付', '22,226.54', '1,116.50', '3,889.64', '3,540.24', '30,772.92', '', ''),
 (' 赔付', '145.

In [70]:
# 3.通过re模块提取数据-对账单序列号
# import re
p=r'Credit Note Number (\d{7})'
res_number=re.findall(p,data)
res_number=res_number[0]
res_number

'1308122'

In [71]:
# 4.通过re模块提取数据-对账单日期
# import re
p=r'created on (\S{10})'
res_date=re.findall(p,data)
res_date=res_date[0]
res_date

'2022.09.21'

In [72]:

# 5.判断国产还是进口
first_char=res_head[0][1][0]
if first_char=='L':
    s='BBA'
else:
    s='GIS'
s

'BBA'

In [73]:
# 通过pandas转换成dataframe
import pandas as pd
import numpy as np
df1=pd.DataFrame(res_head,columns=['保修单号','车架号','DWP保修单号'])
df1
df2=pd.DataFrame(res_body,columns=list('A1234567'))
df2
# 合并两个dataframe concat axis=1
df=pd.concat([df1,df2],axis=1)
df

# 去掉',',并转换成float
df.loc[:,'1':'7']=df.loc[:,'1':'7'].applymap(lambda x:str(x).replace(',',''))

# 转换成float
for i in range(1,8):
    df.loc[:,'{}'.format(i)]=pd.to_numeric(df.loc[:,'{}'.format(i)],errors='coerce')

# 添加汇总列
df['总计']=df.loc[:,'1':'7'].max(axis=1)

# 添加各种'对账单序列号','对账单日期'列
df['对账单序列号']=res_number
df['对账单日期']=res_date.replace('.','/')

df.head()
# df.info()

,保修单号,车架号,DWP保修单号,A,1,2,3,4,5,6,7,总计,对账单序列号,对账单日期
0,212593,LBVHY3107MMZ64537,A5JVU6,赔付,145.14,130.96,459.57,95.64,831.31,NaN,NaN,831.31,1308122,2022/09/21
1,213378,LBVKY9108LSX09222,A5NONU,赔付,139.72,446.60,10.06,77.53,673.91,NaN,NaN,673.91,1308122,2022/09/21
2,213377,LBVKY9109LSW76277,A5NONV,赔付,152.91,446.60,11.01,79.37,689.89,NaN,NaN,689.89,1308122,2022/09/21
3,213379,LBVKY9108LSU91847,A5NONW,赔付,139.72,446.60,10.06,77.53,673.91,NaN,NaN,673.91,1308122,2022/09/21
4,213380,LBVKY5101LSY22048,A5NONX,赔付,27137.40,2424.40,1953.90,4097.04,35612.74,NaN,NaN,35612.74,1308122,2022/09/21



# 附加功能-找到每个细节

In [74]:
df.shape

(76, 14)

In [75]:
# 0.re提取每个特征-生成段落
import re
detail_list=re.findall(r'[^_]+',data)[:-1]
detail_list.__len__()
# detail_list[0]


76

In [76]:
# 1提取工时行
FRU=re.findall(r'(\S{10}) (\d{7}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
FRU
df_FRU=pd.DataFrame(FRU,columns=list('ABCDFHKL'))
df_FRU

,A,B,C,D,F,H,K,L
0,8700060086,0000105,32.74,80.0,2,52.38,6.81,59.19
1,8700020086,0000610,32.74,80.0,3,78.58,10.22,88.80


In [77]:
# 2.提取零件行-正常配件
part_normal=re.findall(r'(\S{10}) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[1])
part_normal
df_part_normal=pd.DataFrame(part_normal,columns=list('ABCDEFGJKL'))
df_part_normal

,A,B,C,D,E,F,G,J,K,L
0,0012740500,12415A77FD6,36.28,100.0,29.00-,1,25.76,1.85,3.59,31.20
1,0012740500,12415A78409,144.25,100.0,21.00-,1,113.96,8.21,15.88,138.05


In [78]:
# 3.提取零件行-BSI配件
part_BSI=re.findall(r'(8\S{8}6) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
part_BSI
df_part_BSI=pd.DataFrame(part_BSI,columns=list('ABCDFGKL'))
df_part_BSI

,A,B,C,D,F,G,K,L
0,8700020086,11428593186,181.42,80.0,1,145.14,18.87,164.01


In [79]:
# 4.提取辅料
sub_4=re.findall(r'(\S{10}) (\S{1}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',detail_list[0])
sub_4
df_sub_4=pd.DataFrame(sub_4,columns=list('ABCDFIKL'))
df_sub_4

,A,B,C,D,F,I,K,L
0,8700020086,4,574.46,80.0,1,459.57,59.74,519.31


In [80]:
# 5.找到单行的表头
p=r'(\d{5,6}) (\S{17}) (\S{6})'
one_head=re.findall(p,detail_list[0])
one_head
claim,vin,dwp=one_head[0]
claim,vin,dwp

('212593', 'LBVHY3107MMZ64537', 'A5JVU6')

In [81]:
# 建立一个空的dataframe，只包括表头-作用是避免漏掉没有数据的列
df_na=pd.DataFrame(columns=list('ABCDEFGHIJKL'))
df_na

,A,B,C,D,E,F,G,H,I,J,K,L


In [82]:
# 拼接dataframe
res_one_part=pd.concat([df_FRU,df_part_normal,df_part_BSI,df_sub_4],axis=0)
# column 按照字母排序
res_one_part.sort_index(axis=1,inplace=True)
# claim vin dwp 插入前三列
res_one_part.insert(0,'claim',claim)
res_one_part.insert(1,'vin',vin)
res_one_part.insert(2,'dwp',dwp)
res_one_part

,claim,vin,dwp,A,B,C,D,E,F,G,H,I,J,K,L
0,212593,LBVHY3107MMZ64537,A5JVU6,8700060086,0000105,32.74,80.0,NaN,2,NaN,52.38,NaN,NaN,6.81,59.19
1,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,0000610,32.74,80.0,NaN,3,NaN,78.58,NaN,NaN,10.22,88.80
0,212593,LBVHY3107MMZ64537,A5JVU6,0012740500,12415A77FD6,36.28,100.0,29.00-,1,25.76,NaN,NaN,1.85,3.59,31.20
1,212593,LBVHY3107MMZ64537,A5JVU6,0012740500,12415A78409,144.25,100.0,21.00-,1,113.96,NaN,NaN,8.21,15.88,138.05
0,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,11428593186,181.42,80.0,NaN,1,145.14,NaN,NaN,NaN,18.87,164.01
0,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,4,574.46,80.0,NaN,1,NaN,NaN,459.57,NaN,59.74,519.31


In [88]:
# 0.re提取每个特征-生成段落
import re
detail_list=re.findall(r'[^_]+',data)[:-1]
detail_list.__len__()

res_list=[]
for i in detail_list:
    
    
    
    # 
    # 1提取工时行
    FRU=re.findall(r'(\S{10}) (\d{7}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    FRU
    df_FRU=pd.DataFrame(FRU,columns=list('ABCDFHKL'))
    df_FRU

    # 
    #   2.提取零件行-正常配件
    part_normal=re.findall(r'(\S{10}) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    part_normal
    df_part_normal=pd.DataFrame(part_normal,columns=list('ABCDEFGJKL'))
    df_part_normal

    # 
    # 3.提取零件行-BSI配件
    part_BSI=re.findall(r'(8\S{8}6) (\S{11}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    part_BSI
    df_part_BSI=pd.DataFrame(part_BSI,columns=list('ABCDFGKL'))
    df_part_BSI

    # 
    # 4.提取辅料
    sub_4=re.findall(r'(\S{10}) (\S{1}) (\S+) (\S+) (\S+) (\S+) (\S+) (\S+)',i)
    sub_4
    df_sub_4=pd.DataFrame(sub_4,columns=list('ABCDFIKL'))
    df_sub_4

    # 
    # 5.找到单行的表头
    p=r'(\d{5,6}) (\S{17}) (\S{6})'
    one_head=re.findall(p,i)
    one_head
    claim,vin,dwp=one_head[0]
    claim,vin,dwp

    # 
    # 建立一个空的dataframe，只包括表头-作用是避免漏掉没有数据的列
    df_na=pd.DataFrame(columns=list('ABCDEFGHIJKL'))
    df_na

    # 
    # 拼接dataframe
    res_one_part=pd.concat([df_FRU,df_part_normal,df_part_BSI,df_sub_4,df_na],axis=0)
    # column 按照字母排序
    res_one_part.sort_index(axis=1,inplace=True)
    # claim vin dwp 插入前三列
    res_one_part.insert(0,'claim',claim)
    res_one_part.insert(1,'vin',vin)
    res_one_part.insert(2,'dwp',dwp)
    res_list.append(res_one_part)

res=pd.concat(res_list,axis=0)
# 将A-L列转换为数值
# res[list('ABCDEFJHIJKL')]=res[list('ABCDEFJHIJKL')].apply(pd.to_numeric,errors='ignore')
res

,claim,vin,dwp,A,B,C,D,E,F,G,H,I,J,K,L
0,212593,LBVHY3107MMZ64537,A5JVU6,8700060086,0000105,32.74,80.0,NaN,2,NaN,52.38,NaN,NaN,6.81,59.19
1,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,0000610,32.74,80.0,NaN,3,NaN,78.58,NaN,NaN,10.22,88.80
0,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,11428593186,181.42,80.0,NaN,1,145.14,NaN,NaN,NaN,18.87,164.01
0,212593,LBVHY3107MMZ64537,A5JVU6,8700020086,4,574.46,80.0,NaN,1,NaN,NaN,459.57,NaN,59.74,519.31
0,213378,LBVKY9108LSX09222,A5NONU,12740500,0072182,31.90,100.0,NaN,14,NaN,446.60,NaN,NaN,58.06,504.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,213423,LBVKY910XLSY21147,A5NWU7,12740500,12415A77FD6,36.28,100.0,29.00-,1,25.76,NaN,NaN,1.85,3.59,31.20
1,213423,LBVKY910XLSY21147,A5NWU7,12740500,12415A78409,144.25,100.0,21.00-,1,113.96,NaN,NaN,8.21,15.88,138.05
0,213422,LBVKY9106LSU93631,A5NWU8,12740500,0072720,31.90,100.0,NaN,12,NaN,382.80,NaN,NaN,49.76,432.56
0,213422,LBVKY9106LSU93631,A5NWU8,12740500,12415A77FD6,36.28,100.0,29.00-,1,25.76,NaN,NaN,1.85,3.59,31.20


In [89]:

res.to_excel('res1.xlsx',index=False)